# Balancer Simulations Math Challenge - Advanced

This notebook provides a collection of challenges for an advanced understanding of Balancer Math.

# Challenge No. 1

For the following questions, there are two tokens $X$ and $Y$, with $x$ representing the number of Token $X$ in the pool, and $y$ representing the number of Token $Y$ in the pool. 

The plots below represent the invariant curves for two Balancer Pools. Both curves are of the form $2 = x^ay^{1-a}$ for some value of $a$. Try as many of the exercises below as you want -- discuss them with others and have fun!

1. Explain how the equation $V = B_1^{W_1}B_2^{W_2}$ becomes $2 = x^ay^{1-a}$ in this context. 

2. Give one possible value of $a$ and a pair of possible legal values for $x$ and $y$ in this context. 

3. Rewrite the curve $2 = x^ay^{1-a}$ in the form "$y =$ (some expression involving x)" and plot it using the Python tool of your choice. 

4. Generate 100 $(x,y)$ points on the curve $2 = x^{0.6}y^{0.4}$. Now take the **logarithm** of both $x$ and $y$ in your 100 points. Plot $\log(x)$ against $\log(y)$. What do you notice? 

5. Both pictures above represent AMM curves of the form $2 = x^{a}y^{1-a}$. One of the curves has $a =0.6$ and the other has $a = 0.8$. Which is which? How can you tell? 

6. Both of the curves pictured above contain the point $(2,2)$. Explain why any curve of the form $2= x^{a}y^{1-a}$ will pass through $(2,2)$. Is this a special property of the number 2, or will any curve of the form $k = x^{a}y^{1-a}$ pass through $(k,k)$? (You can determine this with an algebraic proof, or by playing with Python graphs.)

7. Suppose a curve $2 = x^ay^{1-a}$ passes through the point $(2,2)$ and $(p,q)$. Explain how you can use the values of $p$ and $q$ to determine the weights of Token $X$ and Token $Y$. 

8. In **Alice's Pool**, which point has the highest spot price $SP_X^{Y}$: **A**, **B** or **C**? (You may be able to answer this without calculation.)

9. Suppose Alice is actively managing her pool against Bob's pool and wishes to trade it against Bob's pool. Identify a pair of points -- one in her pool and one in Bob's pool -- that would represent an arbitrage opportunity for Alice. 

10. Create the invariant curves for Alice's pool and Bob's pool in Python using the graphing tool of your choice. Choose one point on Bob's curve and highlight it in Blue. Call this point **Z**. On your graph of Alice's curve, color in red **all of the points that represent arbitrage opportunities for Alice against Z**, i.e. color red all of the points on Alice's curve that represent where she could make a profit if she traded with Bob while he held the position represented by **Z**. (**Note**: This is the hardest one in terms of symbolic math, since it involves solving an inequality involving power functions.)

# Challenge No. 2

Alice wants to set up another pool.  

She knows that the current price of 1 token C is 37 token D.  
She owns 57000 token C, and 510000 token D. She plans to set a fee to 1%, and maximize her capital returns via this pool.  

#### 1.
What is the optimal set up for Alice’s new pool? Think about how you’d approach it and explain why.

#### 2.
Due to a huge price dump, the price of token C on external markets drops to 28 token D. Now, Arbitrage traders get to work. Image Alice wanted to re-balance the pool herself, how much token C would she need in order to re-balance the pool in one trade? Keep in mind swap fees and slippage!

#### 3.
Imagine Alice’s pool would be twice as big. How much liquidity would she need for the same price change?

#### 4.
Explore the general relation between the size of the pool (liquidity m in token C and n in token D) and price changes with a series of experiments.  
	• derive an expression  
	• plot a graph  

# Challenge No. 3

Bob runs a 50-50 Token X-Token Y pool with a 5% swap fee. 

The following traders want to interact with this pool:
- Carlos wants to increase the amount of Token X by 10% by trading X for Y
- Diana wants to increase the amount of Token Y by 10% by trading Y for X
- Ellen wants to increase the spot price X:Y by 10%
- Fabricio wants to increase the spot price Y:X by 10%

#### 1.
For each action, specify the amount that the person will need to trade in to achieve the result. 

#### 2.
These actions can not occur simultaneously and will be processed in some order. There are 24 different total orders -- choose a few different ones and simulate the effect on spot price and liquidity. Can you make a conclusion about how increasing liquidity or changing balances affects spot price? If not, try running additional simulations with varying numbers besides 10%.

#### 3.
Now add in two agents who want to increase the pool liquidity by making a deposit. With 6 actors, there are 720 different orderings of the actions. Analyze some. Which actions work "together" (reducing the amount needed of the second action to achieve a particular effect) and which work "against each other"? 

In [1]:
## NOTES:

### OK--> Create and initialise the pool (x,y,100,100,0.5,0.5), 
### OK--> Initialise balances for actors
### 1. Amount of Tokens X, Y to achieve the result

    # Carlos increase X 10% by trading X for Y --> calc_out_given_in
    # Diana increase Y 10% by trading Y for X --> calc_out_given_in
    # Elen increase spot-price X/Y 10% --> Use formula from whitepapper
    # Fabrizio increase spot-price Y/X 10% --> Use formula from whitepapper
    
### 2. Relation of liquidity and balances on spot-price
### 3. 2 Agents x 6 actors


### **"If weights are held constant, the spot prices offered by Balancer Pools only change with changing token balances. If the pool owner does not add or remove tokens to/from the pool, token balances can only change through trades."**

Source: Balancer Whitepaper(2019), page 4. https://balancer.fi/whitepaper.pdf
<br></br>

In [2]:
pip install balancerv2cad==0.1.91

Note: you may need to restart the kernel to use updated packages.


# 0. Dependencies

In [3]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd

# Additional dependencies

# For analytics
import numpy as np
# For visualization
import plotly.express as px

# For using the balancer pool model
from balancerv2cad.WeightedPool import WeightedPool
from balancerv2cad.WeightedMath import WeightedMath
from balancerv2cad import BalancerConstants as bc

from decimal import Decimal

# 0. Preparatory Steps

**Instantiate a new balancer pool**

In [4]:
bp = WeightedPool()

**Create and initialise the Boob's pool**

With two tokens X and Y equal weights of 50-50 and balances of 100 each. 

In [5]:
bp.join_pool(
    # First argument populates "balances" as a dict with key-value pair 'token' is a string and 'value' as an integer
    {'Token_X':100,
     'Token_Y':100},
    
    # First argument populates "weights" as a dict with key-value pair 'token' is a string and 'value' as float
    {'Token_X':0.5,
     'Token_Y':0.5}
)

In [6]:
print("Balance_of_'Token_X': {}. Balance_of_'Token_Y': {}".format(bp._balances['Token_X'], bp._balances['Token_Y']))
print("Weight_of_Token'Token_X': {}. Weight_of_Token_'Token_Y': {}".format(bp._weights['Token_X'], bp._weights['Token_Y']))

Balance_of_'Token_X': 100. Balance_of_'Token_Y': 100
Weight_of_Token'Token_X': 0.5. Weight_of_Token_'Token_Y': 0.5


**Set the fee for the pool to 5%**

In [7]:
bp._swap_fee = Decimal(0.05)

In [8]:
print(bp._swap_fee)

0.05000000000000000277555756156289135105907917022705078125


In [9]:
bp.factory_fees

{'Token_X': Decimal('0'), 'Token_Y': Decimal('0')}

**Initialise Balances of Actors**

In [10]:
#Balances for Actors -> each actor should own X = 100, Y = 100
balances_carlos = {'X': 100, 'Y': 100}
balances_diana = {'X': 100, 'Y': 100}
balances_ellen = {'X': 100, 'Y': 100}
balances_fabrizio = {'X': 100, 'Y': 100}               

In [11]:
print(balances_carlos)
print(balances_diana)
print(balances_ellen)
print(balances_fabrizio)

{'X': 100, 'Y': 100}
{'X': 100, 'Y': 100}
{'X': 100, 'Y': 100}
{'X': 100, 'Y': 100}


In [12]:
print(balances_carlos['X'])
print(balances_ellen['Y'])

100
100


# 1. 

For each action, specify the amount that the person will need to **trade in** to achieve the result. 

In [13]:
# OK -> 1.1 Carlos: calc amount of X trade-in to achieve 10% increase of X by trading X for Y -> balances X,Y
# TO-DO -> 1.2 Diana: calc amount of Y trade-in to achieve 10% increase of Y by trading Y for X -> balances X,Y
# TO-DO ->  1.3 Ellen: calc amount of X trade-in to achiece 10% increase of spot-price X/Y
# TO-DO ->  1.4 Fabrizio: calc amount of Y tradi-in to achieve 10% increase of spot-price Y/X

## 1.1

Carlos -> calc amount of X trade-in to achieve 10% increase of X by trading X for Y -> balances X,Y

In [14]:
# Initial balances of Token_X in the pool
balance_of_X_pool = bp._balances['Token_X']
balance_of_X_pool

Decimal('100')

In [15]:
# Carlos Initial balances of Token_X
print(balances_carlos)

{'X': 100, 'Y': 100}


In [16]:
# By equal weights amount of Token_ trade-in the pool is derect 
# propotional to the balance of Token_X in the pool
amount_of_Token_X = 10

#Update balances of Token_X for the pool and Bob
balance_of_X_pool += bp.swap('Token_X', 'Token_Y', amount_of_Token_X, given_in=True )
balances_carlos['X'] -= amount_of_Token_X 

In [38]:
# Print out results fo balances of Token_X and Token_Y in the pool
print("Balance of 'Token_X in the pool': {}.".format([bp._balances['Token_X']]))
print("Balance of 'Token_Y in the pool': {}".format([bp._balances['Token_Y']]))
# Print out results fo balances of Token_X and Token_Y in the pool
print("Bob's Balance of 'Token_X ':{}.". format([balances_carlos['X']]))

Balance of 'Token_X in the pool': [Decimal('119.9025000000000091784913002')].
Balance of 'Token_Y in the pool': [Decimal('83.40109672442191976400624907')]
Bob's Balance of 'Token_X ':[90].


In [18]:
# Update weights of Token_X and Token_Y after swap
weight_of_Token_X = bp._weights['Token_X']
updated_weight_of_Token_X = bp._balances['Token_X']/(bp._balances['Token_X'] + bp._balances['Token_Y'])
updated_weight_of_Token_Y = bp._balances['Token_Y']/(bp._balances['Token_X'] + bp._balances['Token_Y'])

In [36]:
# Print out updated weights of Token_X and Token_Y after swap
print("Weight_of_Token'Token_X': {}".format(updated_weight_of_Token_X, updated_weight_of_Token_Y))
print("Weight_of_Token_'Token_Y': {}".format(updated_weight_of_Token_Y))

Weight_of_Token'Token_X': 0.5452530098566410112316064427
Weight_of_Token_'Token_Y': 0.4547469901433589887683935572


In [37]:
check = updated_weight_of_Token_X + updated_weight_of_Token_Y
print(check)

0.9999999999999999999999999999


In [21]:
bp.factory_fees

{'Token_X': Decimal('0.5000000000000000277555756156'), 'Token_Y': Decimal('0')}

## 1.2

Diana: calc amount of Y trade-in to achieve 10% increase of Y by trading Y for X -> balances X,Y

In [48]:
bp = WeightedPool()

In [49]:
bp.join_pool(
    # First argument populates "balances" as a dict with key-value pair 'token' is a string and 'value' as an integer
    {'Token_Y':100,
     'Token_X':100},
    
    # First argument populates "weights" as a dict with key-value pair 'token' is a string and 'value' as float
    {'Token_Y':0.5,
     'Token_X':0.5}
)

In [50]:
print("Balance_of_'Token_Y': {}. Balance_of_'Token_X': {}".format(bp._balances['Token_Y'], bp._balances['Token_X']))
print("Weight_of_Token'Token_Y': {}. Weight_of_Token_'Token_X': {}".format(bp._weights['Token_Y'], bp._weights['Token_X']))

Balance_of_'Token_Y': 100. Balance_of_'Token_X': 100
Weight_of_Token'Token_Y': 0.5. Weight_of_Token_'Token_X': 0.5


In [51]:
bp._swap_fee = Decimal(0.05)
print(bp._swap_fee)

0.05000000000000000277555756156289135105907917022705078125


In [52]:
bp.factory_fees

{'Token_Y': Decimal('0'), 'Token_X': Decimal('0')}

In [53]:
#Balances for Actors -> each actor should own X = 100, Y = 100
balances_carlos = {'X': 100, 'Y': 100}
balances_diana = {'X': 100, 'Y': 100}
balances_ellen = {'X': 100, 'Y': 100}
balances_fabrizio = {'X': 100, 'Y': 100}   

print(balances_carlos)
print(balances_diana)
print(balances_ellen)
print(balances_fabrizio)

{'X': 100, 'Y': 100}
{'X': 100, 'Y': 100}
{'X': 100, 'Y': 100}
{'X': 100, 'Y': 100}


In [54]:
# Diana increase Y 10% by trading Y for X --> calc_out_given_in
balance_of_Y_pool = bp._balances['Token_X']
print(balance_of_Y_pool)
print(balances_diana)

100
{'X': 100, 'Y': 100}


In [55]:
# By equal weights amount of Token_ trade-in the pool is derect 
# propotional to the balance of Token_Y in the pool
amount_of_Token_Y = 10

#Update balances of Token_Y for the pool and Bob
balance_of_Y_pool += bp.swap('Token_Y', 'Token_X', amount_of_Token_Y, given_in=True )
balances_carlos['Y'] -= amount_of_Token_Y 

In [56]:
# Print out results fo balances of Token_X and Token_Y in the pool
print("Balance of 'Token_Y in the pool': {}.".format([bp._balances['Token_Y']]))
print("Balance of 'Token_X in the pool': {}".format([bp._balances['Token_X']]))
# Print out results fo balances of Token_X and Token_Y in the pool
print("Bob's Balance of 'Token_Y ':{}.". format([balances_diana['Y']]))

Balance of 'Token_Y in the pool': [Decimal('109.4999999999999999722444243')].
Balance of 'Token_X in the pool': [Decimal('91.32420091324200915556854578')]
Bob's Balance of 'Token_Y ':[100].


In [58]:
weight_of_Token_Y = bp._weights['Token_Y']
updated_weight_of_Token_Y = bp._balances['Token_Y']/(bp._balances['Token_Y'] + bp._balances['Token_X'])
updated_weight_of_Token_X = bp._balances['Token_X']/(bp._balances['Token_Y'] + bp._balances['Token_X'])

In [59]:
# Print out updated weights of Token_X and Token_Y after swap
print("Weight_of_Token'Token_Y': {}".format(updated_weight_of_Token_Y, updated_weight_of_Token_X))
print("Weight_of_Token_'Token_X': {}".format(updated_weight_of_Token_X))

Weight_of_Token'Token_Y': 0.5452530098566410112316064427
Weight_of_Token_'Token_X': 0.4547469901433589887683935572


In [60]:
bp.factory_fees

{'Token_Y': Decimal('0.5000000000000000277555756156'), 'Token_X': Decimal('0')}

## 1.3

Ellen: calc amount of X trade-in to achiece 10% increase of spot-price X/Y

In [22]:
# Calculate after-trade effective price (EF) 

EP_11 = bp._balances['Token_X']/bp._balances['Token_Y']
print(EP_11)

1.199024999999999999392152893


In [23]:
# Calculate the new SP
SP_13 = Decimal(1.1)* Decimal(EP_11)
SP_13 = Decimal(SP_13)
print(SP_13)

1.318927500000000105826181150


In [24]:
# Calculate additional amount of Token_X for icreasing SP

balance_of_Token_X_13 = SP_13*(bp._balances['Token_X'] * updated_weight_of_Token_Y)/updated_weight_of_Token_X
amount_of_Token_X_for_13 = balance_of_Token_X_13 - bp._balances['Token_X']

# Calculate swap_fee for the amount_of-Token_X_for_13 and add to the amount_of-Token_X_for_13 
swap_fee_for_13 = Decimal(0.05) * amount_of_Token_X_for_13
swap_fee_for_13 = Decimal(swap_fee_for_13)
Amount_of_Token_X_for_13_wFee = swap_fee_for_13 + amount_of_Token_X_for_13

print("Balance of 'Token_X_13': {}".format(balance_of_Token_X_13))
print("Αmount of 'Token_X_13' for SP_13: {}".format(amount_of_Token_X_for_13))
print("Swap Fee for Amount of Token_X_13' for SP_13: {}".format(swap_fee_for_13))
print("Amount of Token_X_13' for SP_13 with Fees: {}".format(Amount_of_Token_X_for_13_wFee))

Balance of 'Token_X_13': 120.4500000000000096950225624
Αmount of 'Token_X_13' for SP_13: 10.9500000000000097227781381
Swap Fee for Amount of Token_X_13' for SP_13: 0.5475000000000005165312622041
Amount of Token_X_13' for SP_13 with Fees: 11.49750000000001023930940030


In [25]:
#Make the swap, update balances of Token_X for the pool and Ellen

balance_of_Token_X_13 += bp.swap('Token_X', 'Token_Y', amount_of_Token_X_for_13, given_in=True )
balances_ellen['X'] -= amount_of_Token_X_for_13 

In [35]:
# Print out results fo balances of Token_X and Token_Y in the pool
print("Balance of 'Token_X in the pool':{}". format([bp._balances['Token_X']]))
print("Balance of 'Token_Y in the pool':{}". format([bp._balances['Token_Y']]))
# Print out results fo balances of Token_X and Token_Y in the pool
print("Ellen's Balance of 'Token_X ':{}". format([balances_ellen['X']]))

Balance of 'Token_X in the pool':[Decimal('119.9025000000000091784913002')]
Balance of 'Token_Y in the pool':[Decimal('83.40109672442191976400624907')]
Ellen's Balance of 'Token_X ':[Decimal('89.0499999999999902772218619')]


In [28]:
# Calculate after-trade effective price (EF) 

EP_13 = bp._balances['Token_X']/bp._balances['Token_Y']
print(EP_13)

1.437660950625000220104810626


## 1.4

In [ ]:
# Fabrizio: calc amount of Y tradi-in to achieve 10% increase of spot-price Y/X

In [62]:
effective_price = bp._balances['Token_Y']/bp._balances['Token_X']
spot_price = Decimal(1.1)*Decimal(effective_price)
spot_price = Decimal(spot_price)
print(spot_price)
print(effective_price)

1.318927500000000105826181150
1.199024999999999999392152893


In [63]:
balance_of_Token_Y_fab = spot_price*(bp._balances['Token_Y'] * updated_weight_of_Token_X)/updated_weight_of_Token_X
amount_of_Token_Y_for_fab = balance_of_Token_Y_fab - bp._balances['Token_Y']

# Calculate swap_fee for the amount_of-Token_X_for_13 and add to the amount_of-Token_X_for_13 
swap_fee_for_fab = Decimal(0.05) * amount_of_Token_Y_for_fab
swap_fee_for_fab = Decimal(swap_fee_for_fab)
Amount_of_Token_Y_for_fab_with_Fee = swap_fee_for_fab + amount_of_Token_Y_for_fab

print("Balance of 'Token_Y_fab': {}".format(balance_of_Token_Y_fab))
print("Αmount of 'Token_Y_fab' for spot_price: {}".format(amount_of_Token_Y_for_fab))
print("Swap Fee for Amount of Token_Y_fab' for spot_price: {}".format(swap_fee_for_fab))
print("Amount of Token_Y_fab' for spot_price with Fees: {}".format(Amount_of_Token_Y_for_fab_with_Fee))

Balance of 'Token_Y_fab': 144.4225612500000115513592438
Αmount of 'Token_Y_fab' for spot_price: 34.9225612500000115791148195
Swap Fee for Amount of Token_Y_fab' for spot_price: 1.746128062500000675885319922
Amount of Token_Y_fab' for spot_price with Fees: 36.66868931250001225500013942


In [64]:
#Make the swap, update balances of Token_X for the pool and Ellen

balance_of_Token_Y_fab += bp.swap('Token_Y', 'Token_X', amount_of_Token_Y_for_fab, given_in=True )
balances_ellen['Y'] -= amount_of_Token_Y_for_fab 

In [65]:
# Print out results fo balances of Token_X and Token_Y in the pool
print("Balance of 'Token_Y in the pool':{}". format([bp._balances['Token_Y']]))
print("Balance of 'Token_X in the pool':{}". format([bp._balances['Token_X']]))
# Print out results fo balances of Token_X and Token_Y in the pool
print("Fabrizio's Balance of 'Token_Y ':{}". format([balances_ellen['Y']]))

Balance of 'Token_Y in the pool':[Decimal('142.6764331875000108754739238')]
Balance of 'Token_X in the pool':[Decimal('70.08865989002104859443507665')]
Fabrizio's Balance of 'Token_Y ':[Decimal('65.0774387499999884208851805')]


In [66]:
effective_price = bp._balances['Token_Y']/bp._balances['Token_X']
print(effective_price)

2.035656458710715451350390737


# 2. 

These actions can not occur simultaneously and will be processed in some order. There are 24 different total orders -- choose a few different ones and simulate the **effect on spot price and liquidity**. Can you make a conclusion about **how increasing liquidity or changing balances affects spot price**? If not, try running additional simulations with varying numbers besides 

# Create Permutation

In [43]:
# See https://stackoverflow.com/a/34032549
import itertools

Actors = ['Carlos', 'Diana', 'Ellen', 'Fabrizio']
    
trading_scenario = list(itertools.permutations(Actors))
print(trading_scenario)

trading_scenario = [x[0] for x in trading_scenario]

combinations = {
    'Actors': trading_scenario 
}

Amount_combinations = len(trading_scenario)

print(Amount_combinations)


[('Carlos', 'Diana', 'Ellen', 'Fabrizio'), ('Carlos', 'Diana', 'Fabrizio', 'Ellen'), ('Carlos', 'Ellen', 'Diana', 'Fabrizio'), ('Carlos', 'Ellen', 'Fabrizio', 'Diana'), ('Carlos', 'Fabrizio', 'Diana', 'Ellen'), ('Carlos', 'Fabrizio', 'Ellen', 'Diana'), ('Diana', 'Carlos', 'Ellen', 'Fabrizio'), ('Diana', 'Carlos', 'Fabrizio', 'Ellen'), ('Diana', 'Ellen', 'Carlos', 'Fabrizio'), ('Diana', 'Ellen', 'Fabrizio', 'Carlos'), ('Diana', 'Fabrizio', 'Carlos', 'Ellen'), ('Diana', 'Fabrizio', 'Ellen', 'Carlos'), ('Ellen', 'Carlos', 'Diana', 'Fabrizio'), ('Ellen', 'Carlos', 'Fabrizio', 'Diana'), ('Ellen', 'Diana', 'Carlos', 'Fabrizio'), ('Ellen', 'Diana', 'Fabrizio', 'Carlos'), ('Ellen', 'Fabrizio', 'Carlos', 'Diana'), ('Ellen', 'Fabrizio', 'Diana', 'Carlos'), ('Fabrizio', 'Carlos', 'Diana', 'Ellen'), ('Fabrizio', 'Carlos', 'Ellen', 'Diana'), ('Fabrizio', 'Diana', 'Carlos', 'Ellen'), ('Fabrizio', 'Diana', 'Ellen', 'Carlos'), ('Fabrizio', 'Ellen', 'Carlos', 'Diana'), ('Fabrizio', 'Ellen', 'Diana', 'C

# 3. 

Now add in two agents who want to increase the pool liquidity by making a deposit. With 6 actors, there are 720 different orderings of the actions. Analyze some. **Which actions work "together"** (reducing the amount needed of the second action to achieve a particular effect) and **which work "against each other"**? 

# Create Permutation agent 1

In [41]:
# See https://stackoverflow.com/a/34032549
import itertools

agent1 = ['A', 'B', 'C', 'D', 'E','F']
    
trading_scenario = list(itertools.permutations(Agent1))
print(trading_scenario)

trading_scenario = [x[0] for x in trading_scenario]

combinations = {
    'Actors': trading_scenario 
}

Amount_combinations = len(trading_scenario)

print(Amount_combinations)

[('A', 'B', 'C', 'D', 'E', 'F'), ('A', 'B', 'C', 'D', 'F', 'E'), ('A', 'B', 'C', 'E', 'D', 'F'), ('A', 'B', 'C', 'E', 'F', 'D'), ('A', 'B', 'C', 'F', 'D', 'E'), ('A', 'B', 'C', 'F', 'E', 'D'), ('A', 'B', 'D', 'C', 'E', 'F'), ('A', 'B', 'D', 'C', 'F', 'E'), ('A', 'B', 'D', 'E', 'C', 'F'), ('A', 'B', 'D', 'E', 'F', 'C'), ('A', 'B', 'D', 'F', 'C', 'E'), ('A', 'B', 'D', 'F', 'E', 'C'), ('A', 'B', 'E', 'C', 'D', 'F'), ('A', 'B', 'E', 'C', 'F', 'D'), ('A', 'B', 'E', 'D', 'C', 'F'), ('A', 'B', 'E', 'D', 'F', 'C'), ('A', 'B', 'E', 'F', 'C', 'D'), ('A', 'B', 'E', 'F', 'D', 'C'), ('A', 'B', 'F', 'C', 'D', 'E'), ('A', 'B', 'F', 'C', 'E', 'D'), ('A', 'B', 'F', 'D', 'C', 'E'), ('A', 'B', 'F', 'D', 'E', 'C'), ('A', 'B', 'F', 'E', 'C', 'D'), ('A', 'B', 'F', 'E', 'D', 'C'), ('A', 'C', 'B', 'D', 'E', 'F'), ('A', 'C', 'B', 'D', 'F', 'E'), ('A', 'C', 'B', 'E', 'D', 'F'), ('A', 'C', 'B', 'E', 'F', 'D'), ('A', 'C', 'B', 'F', 'D', 'E'), ('A', 'C', 'B', 'F', 'E', 'D'), ('A', 'C', 'D', 'B', 'E', 'F'), ('A', '

In [44]:
# See https://stackoverflow.com/a/34032549
import itertools

agent2 = ['G', 'H', 'I', 'J', 'K','L']
    
trading_scenario = list(itertools.permutations(Agent1))
print(trading_scenario)

trading_scenario = [x[0] for x in trading_scenario]

combinations = {
    'Actors': trading_scenario 
}

Amount_combinations = len(trading_scenario)

print(Amount_combinations)

[('A', 'B', 'C', 'D', 'E', 'F'), ('A', 'B', 'C', 'D', 'F', 'E'), ('A', 'B', 'C', 'E', 'D', 'F'), ('A', 'B', 'C', 'E', 'F', 'D'), ('A', 'B', 'C', 'F', 'D', 'E'), ('A', 'B', 'C', 'F', 'E', 'D'), ('A', 'B', 'D', 'C', 'E', 'F'), ('A', 'B', 'D', 'C', 'F', 'E'), ('A', 'B', 'D', 'E', 'C', 'F'), ('A', 'B', 'D', 'E', 'F', 'C'), ('A', 'B', 'D', 'F', 'C', 'E'), ('A', 'B', 'D', 'F', 'E', 'C'), ('A', 'B', 'E', 'C', 'D', 'F'), ('A', 'B', 'E', 'C', 'F', 'D'), ('A', 'B', 'E', 'D', 'C', 'F'), ('A', 'B', 'E', 'D', 'F', 'C'), ('A', 'B', 'E', 'F', 'C', 'D'), ('A', 'B', 'E', 'F', 'D', 'C'), ('A', 'B', 'F', 'C', 'D', 'E'), ('A', 'B', 'F', 'C', 'E', 'D'), ('A', 'B', 'F', 'D', 'C', 'E'), ('A', 'B', 'F', 'D', 'E', 'C'), ('A', 'B', 'F', 'E', 'C', 'D'), ('A', 'B', 'F', 'E', 'D', 'C'), ('A', 'C', 'B', 'D', 'E', 'F'), ('A', 'C', 'B', 'D', 'F', 'E'), ('A', 'C', 'B', 'E', 'D', 'F'), ('A', 'C', 'B', 'E', 'F', 'D'), ('A', 'C', 'B', 'F', 'D', 'E'), ('A', 'C', 'B', 'F', 'E', 'D'), ('A', 'C', 'D', 'B', 'E', 'F'), ('A', '

# Challenge No. 4

A user has $a$ **TKNA** and $b$ **TKNB**: 
The TKNA-TKNB pool has 50:50 weights, and reserves of TKNA and TKNB of m and n, respectively.  
The user wishes to perform a swap such that the ratio of TKNA:TKNB in their wallet is p:q 

$$\frac{a + x}{b + y} = \frac{p}{q}$$

#### 1. 
Assuming the user performs a swap between TKNA and TKNB on the TKNA-TKNB pool, derive an expression for x and y.

#### 2.
Using data or algebra, which is the best option to reduce slippage when trading X for Y:  
    a) finding a pool with the same invariant but a different X balance value,  
    b) finding a pool with a different weight on X,  
    c) or finding a pool with a different invariant value?  
    Justify your answer in any way you like,  
    and explain whether "different" should be "smaller" or "larger" here.

# Challenge No. 5

#### 1.
**Design a new value function.** 
Use the balances of token A from Q1.2, and apply your new value function.  
Add the resulting balances for token B

#### 2.
**Plot your curve.** 
Send a screenshot of your curve to the Discord channel.  
Let your fellow TE's reverse engineer the value function!

#### 3.
**Pick a screenshot at the Discord.** 
Reverse engineer the value function!